# nilmtkを用いた電力消費を分析する機械学習

## 1. 準備
インドで計測されたデータセットと田中の家で計測した3つの家電(冷蔵庫、テレビ、エアコン)のデータセットを用います。

nilmtkの環境構築は済んでいるものとします。環境構築についてはhttps://docs.google.com/document/d/1lqsn3UlNLvOTVsLZisnizys29YpQzRTqvJiWQcEtpKg/edit?usp=sharing を参照してください。

必要なもの
- electricity_India
- electricity_Tanaka
- metadata_India
- metadata_Tanaka

新しくデータセットを作る場合にはelectricity_〇〇とmetadata_〇〇を1セット準備する必要があります。


- electricity_India：iawe( https://iawe.github.io/ )から"electricity.tar.gz"をダウンロードします。ですが、このままだと全家電対象になってしまうので、ファイルを改変して家電を3つに限定しました。
- electricity_Tanaka：田中の家で計測したデータです。
- metadata_India:インドのデータセットの基本設定を記したものです。
- metadata_Tanaka:田中家のデータセットの基本設定を記したものです。

electricity_〇〇は1.csv～4.csv, labels.datがあります。1.csvが合計の電力、2.csvが冷蔵庫、3.csvがエアコン、4.csvがテレビです。ライブラリの仕様上計測日時をタイムスタンプに変更しています。labels.datに各csvファイルの家電の種類が書いてありますが、datファイルはなくても大丈夫なはずです。

metadata_〇〇はcsvファイルをh5ファイルに変換するために必要な情報です。dataset.yml, meter_devices.yml, building〇.yml, があります。dataset.ymlはデータセットの製作者やデータを取った場所などの基本的な情報を入れます。meta_device.ymlには計測器の情報を入れます。building〇.ymlは建物の数だけ用意します。1個目の建物はbuilding1.yml、2個目の建物はbuilding2.ymlという風になります。building〇.ymlファイルには各家電の種類や親子関係などを記載します。この際家電につけたmeterの番号とcsvファイルの数字を一致させる必要があります。例えば冷蔵庫のデータを2.csvにいれてありますが、building1.ymlでは"type: fridge, meters: [2]"としてあります。説明が難しいので詳しい情報はhttps://nilm-metadata.readthedocs.io/en/latest/tutorial.html のexampleを見てください。

ここから家電を増やしたい場合は
①csvファイルを追加する
②(datファイルも書き換える)
③building〇.ymlを書き換える
とすればよいです。

## 2. CSVファイルとyamlファイルをh5ファイルに変換する
まずは変換する関数を定義します。本当はインドのデータセットについてはライブラリを使えば一発なのですが、家電の種類を3つに揃えるためにライブラリを改造しました。
electiricity_〇〇のフォルダが置いてある場所や家電の数によって一部コードを書き換えます。

In [1]:
from nilmtk.measurement import LEVEL_NAMES
from nilmtk.utils import check_directory_exists, get_datastore, get_module_directory
from nilm_metadata import convert_yaml_to_hdf5
from copy import deepcopy
import datetime

def reindex_fill_na(df, idx):
    df_copy = deepcopy(df)
    df_copy = df_copy.reindex(idx)

    power_columns = [
        x for x in df.columns if x[0] in ['power']]
    non_power_columns = [x for x in df.columns if x not in power_columns]

    for power in power_columns:
        df_copy[power].fillna(0, inplace=True)
    for measurement in non_power_columns:
        df_copy[measurement].fillna(df[measurement].median(), inplace=True)

    return df_copy


column_mapping = {
    'frequency': ('frequency', ""),
    'voltage': ('voltage', ""),
    'W': ('power', 'active'),
    'energy': ('energy', 'apparent'),
    'A': ('current', ''),
    'reactive_power': ('power', 'reactive'),
    'apparent_power': ('power', 'apparent'),
    'power_factor': ('pf', ''),
    'PF': ('pf', ''),
    'phase_angle': ('phi', ''),
    'VA': ('power', 'apparent'),
    'VAR': ('power', 'reactive'),
    'VLN': ('voltage', ""),
    'V': ('voltage', ""),
    'f': ('frequency', "")
}

TIMESTAMP_COLUMN_NAME = "timestamp"
TIMEZONE = "Asia/Tokyo"
START_DATETIME, END_DATETIME = '2023-01-08', '2023-01-09'
FREQ = "1T"

def convert_house(electricity_path, metadata_path, output_filename, format="HDF"):
    """
    Parameters
    ----------
    iawe_path : str
        The root path of the iawe dataset.
    output_filename : str
        The destination filename (including path and suffix).
    """

    check_directory_exists(electricity_path)
    idx = pd.date_range(start=START_DATETIME, end=END_DATETIME, freq=FREQ)
    idx = idx.tz_localize('GMT').tz_convert(TIMEZONE)

    # Open data store
    store = get_datastore(output_filename, format, mode='w')

    # Mains data
    # rangeを変えることでcsvファイルを参照する数を変えます。
    for chan in range(1, 5):
        key = Key(building=1, meter=chan)
        filename = join(electricity_path, "%d.csv" % chan)
        print('Loading ', chan)
        df = pd.read_csv(filename, dtype=np.float64, na_values='\\N')
        df.drop_duplicates(subset=["timestamp"], inplace=True)
        df.index = pd.to_datetime(df.timestamp.values, unit='s', utc=True)
        df = df.tz_convert(TIMEZONE)
        df = df.drop(TIMESTAMP_COLUMN_NAME, 1)
        df.columns = pd.MultiIndex.from_tuples(
            [column_mapping[x] for x in df.columns],
            names=LEVEL_NAMES
        )
        df = df.apply(pd.to_numeric, errors='ignore')
        df = df.dropna()
        df = df.astype(np.float32)
        df = df.sort_index()
        df = df.resample("1T").mean()
        df = reindex_fill_na(df, idx)
        assert df.isnull().sum().sum() == 0
        store.put(str(key), df)
    store.close()
    
    convert_yaml_to_hdf5(metadata_dir, output_filename)

    print("Done converting csv and yaml files to HDF5!")

タイムゾーン、計測開始時と終了時(世界標準時なことに注意)など設定してh5ファイルに変換します。

今回はzipファイルにh5ファイルを同封してあるので実際はこの作業は不要なのですが、一旦h5ファイルを消してみて動くか確認してみてください。

In [ ]:
TIMESTAMP_COLUMN_NAME = "timestamp"
TIMEZONE = "Asia/Tokyo"
START_DATETIME, END_DATETIME = '2023-01-08', '2023-01-09'
FREQ = "1T"

#自分のelectricity_○○とmetadata_〇〇が置いてあるフォルダのパスを参照してください。
electricity_path = "C:/Users/MEIP-users/nilmtk/electricity_Tanaka"
metadata_path = "C:/Users/MEIP-users/nilmtk/metadata_Tanaka"
convert_house(electricity_path, metadata_path, "data_Tanaka.h5")


TIMESTAMP_COLUMN_NAME = "timestamp"
TIMEZONE = "Asia/Kolkata"
START_DATETIME, END_DATETIME = '2013-07-13', '2013-08-04'
FREQ = "1T"

#自分のelectricity_○○とmetadata_〇〇が置いてあるフォルダのパスを参照してください。
electricity_path = "C:/Users/MEIP-users/nilmtk/electricity_India"
metadata_path = "C:/Users/MEIP-users/nilmtk/metadata_India"
convert_house(electricity_path, metadata_path, "data_India.h5")

## 3. 機械学習を実行する
まずは学習データとテストデータをh5ファイルで指定します。

In [ ]:
from nilmtk import DataSet
from nilmtk.utils import print_dict

#学習データとテストデータを指定
train = DataSet('data_India.h5')
test = DataSet('data_Tanaka.h5')
#建物1の電力のデータだけ使う
train_elec = train.buildings[1].elec
test_elec = test.buildings[1].elec

次に機械学習のモデルを作る関数を書きます。

In [ ]:
from __future__ import print_function, division
import time

from matplotlib import rcParams
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from six import iteritems

from nilmtk import DataSet, TimeFrame, MeterGroup, HDFDataStore
import nilmtk.utils

#使用する機械学習のアルゴリズムをインポートします。
from nilmtk.legacy.disaggregate import CombinatorialOptimisation, FHMM

%matplotlib inline

def predict(clf, test_elec, sample_period, timezone):
    pred = {}
    gt= {}
    
    # "ac_type" varies according to the dataset used. 
    # Make sure to use the correct ac_type before using the default parameters in this code.    
    for i, chunk in enumerate(test_elec.mains().load(physical_quantity = 'power', ac_type = 'active', sample_period=sample_period)):
        chunk_drop_na = chunk.dropna()
        pred[i] = clf.disaggregate_chunk(chunk_drop_na)
        gt[i]={}

        for meter in test_elec.submeters().meters:
            # Only use the meters that we trained on (this saves time!)    
            gt[i][meter] = next(meter.load(physical_quantity = 'power', ac_type = 'active', sample_period=sample_period))
        gt[i] = pd.DataFrame({k:v.squeeze() for k,v in iteritems(gt[i]) if len(v)}, index=next(iter(gt[i].values())).index).dropna()
        
    # If everything can fit in memory
    gt_overall = pd.concat(gt)
    gt_overall.index = gt_overall.index.droplevel()
    pred_overall = pd.concat(pred)
    pred_overall.index = pred_overall.index.droplevel()

    # Having the same order of columns
    gt_overall = gt_overall[pred_overall.columns]
    
    #Intersection of index
    gt_index_utc = gt_overall.index.tz_convert("UTC")
    pred_index_utc = pred_overall.index.tz_convert("UTC")
    common_index_utc = gt_index_utc.intersection(pred_index_utc)
    
    common_index_local = common_index_utc.tz_convert(timezone)
    gt_overall = gt_overall.loc[common_index_local]
    pred_overall = pred_overall.loc[common_index_local]
    appliance_labels = [m for m in gt_overall.columns.values]
    gt_overall.columns = appliance_labels
    pred_overall.columns = appliance_labels
    return gt_overall, pred_overall

#機械学習のアルゴリズムを指定。
classifiers = {'FHMM':FHMM()}

predictions = {}

#サンプルタイムの周期を指定します。周期が短いほど学習時間は長くなります。
sample_period = 1

for clf_name, clf in classifiers.items():
    print("*"*20)
    print(clf_name)
    print("*" *20)
    start = time.time()
    # Note that we have given the sample period to downsample the data to 1 minute. 
    # If instead of top_5 we wanted to train on all appliance, we would write 
    # fhmm.train(train_elec, sample_period=60)
    clf.train(train_elec.submeters(), sample_period=sample_period)
    end = time.time()
    print("Runtime =", end-start, "seconds.")
    gt, predictions[clf_name] = predict(clf, test_elec, sample_period, "Asia/Tokyo")

最後に学習モデルをテストデータに当てはめた結果を確認します。結果をCSVファイルに出力します。

In [ ]:
#最初の数行を確認
appliance_labels = [m.label() for m in gt.columns.values]
predictions['FHMM'].columns = appliance_labels
predictions['FHMM'].head()

In [ ]:
#分類結果を図示
predictions['FHMM'].plot(label="Pred")
plt.legend()

In [ ]:
#合計の電力消費を図示
test_elec.mains().plot()

In [ ]:
#CSVファイルを出力
df = predictions['FHMM']
df.to_csv("output.csv")

他にも色々確認できる機能がありますが、確認したくなったらgithub( https://nilmtk.github.io/ )に色々情報が載っているので見てみてください。